<a href="https://colab.research.google.com/github/OculusMode/ILL/blob/main/experimental/Subspaces_with_ILL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from numpy import random
from numpy import linalg as LA

In [3]:
N = 10
while True:
  random_matrix = np.random.random((N, N))
  random_basis, _ = LA.qr(random_matrix)
  if LA.matrix_rank(random_basis) == N:
    break
random_basis

array([[-0.40021388,  0.22366225,  0.07284781,  0.51727684,  0.10740444,
        -0.14276293,  0.54846353, -0.21235634,  0.0890686 , -0.36216609],
       [-0.10626057, -0.02291417,  0.04847707, -0.47336396,  0.75094402,
        -0.3202785 , -0.05896439, -0.09355285, -0.0952046 , -0.2719781 ],
       [-0.16680541, -0.52149557, -0.31076014, -0.19036591, -0.19817107,
        -0.03069509,  0.39817571, -0.1891792 , -0.57442238,  0.05384775],
       [-0.45212247,  0.20641052, -0.15707522, -0.40254575, -0.00809491,
         0.60470104,  0.18908597,  0.36581508,  0.13250043, -0.1157793 ],
       [-0.12303786,  0.02791733,  0.41392633, -0.17703145,  0.0440316 ,
         0.36449034,  0.02003587, -0.7258869 ,  0.09454532,  0.33220947],
       [-0.07100798, -0.02145305,  0.5744789 , -0.24824398, -0.14875789,
        -0.37370806,  0.41136944,  0.38233537,  0.07458401,  0.34654294],
       [-0.0231733 , -0.43583613, -0.11677011, -0.24446166, -0.2844171 ,
        -0.17930951,  0.02585438, -0.17847522